## genome spot

In [58]:
import os
import pandas as pd
from glob import glob

#path to file
input_dir = "genomespot" 
output_file = "genomespot_combined.tsv"

files = glob(os.path.join(input_dir, "*.genomespot.predictions.tsv"))

records = []
for file in files:
    accession = os.path.basename(file).split(".")[0]  # strips file name, gives only accession
    df = pd.read_csv(file, sep="\t")
    # convert the 'value' column into a dictionary keyed by 'target'
    data = df.set_index("target")["value"].to_dict()
    data["accession"] = accession
    records.append(data)

# combine into a single df
combined_df = pd.DataFrame(records)

# reorder columns (accession first)
combined_df = combined_df.set_index('accession').reset_index()

combined_df.head()

,accession,oxygen,ph_max,ph_min,ph_optimum,salinity_max,salinity_min,salinity_optimum,temperature_max,temperature_min,temperature_optimum
0,GCA_903884505,not tolerant,8.986309626449918,5.185028854107618,6.697489401360779,4.384593114378452,0,2.884735866415467,51.93504398348311,35.292992726330255,44.83760079640768
1,GCA_029259185,not tolerant,8.394126703097054,5.517740753819724,6.682644242805267,5.3769576499020255,0,0,73.63443560907696,44.69383883659042,67.8920121656165
2,GCA_000145985,not tolerant,8.204266073484693,4.734636137575809,6.476705995017433,0,0,0,91.20769413444867,73.4988172143595,86.60778627725867
3,GCA_016927425,not tolerant,9.047247843724913,5.373528231259606,6.923415464233607,9.53042418889688,0,2.43275856665094,50.309493452094245,25.424505906619643,45.308049681186404
4,GCF_029338575,tolerant,9.695084118787928,6.028665727235012,7.722688012206665,25.656326155832364,9.384982545542034,15.977666419179668,56.57626924597822,29.58442446319874,46.352929453610784


In [59]:
#save file as tsv
combined_df.to_csv("genomespot_all.tsv", sep="\t", index=False)

In [60]:
#extract OGT 

columns = ["accession", "temperature_optimum"]
spot_df = combined_df[columns] 
spot_df.head()

,accession,temperature_optimum
0,GCA_903884505,44.83760079640768
1,GCA_029259185,67.8920121656165
2,GCA_000145985,86.60778627725867
3,GCA_016927425,45.308049681186404
4,GCF_029338575,46.352929453610784


## genome size

In [61]:
# corrected genome size = assembly length / completeness

import pandas as pd
mi = pd.read_csv("/Users/tamannatandon/Desktop/summer_project/micomplete_out/micomplete_results.tsv", sep="\t", comment="#")

mi.head()

,Name,Length,GC-content,Present Markers,Completeness,Redundancy,Weighted completeness,Weighted redundancy,Contigs,N50,L50,N90,L90,CDs,Unnamed: 14
0,GCF_0005138551_ASM51385v1_genomic,1307099,45.16,126,0.9618,1.0079,0.9651,1.0125,1,1307099,1,1307099,1,1386,NaN
1,GCA_0388711751_ASM3887117v1_genomic,1717258,41.66,117,0.8931,1.0256,0.8529,1.0441,80,29394,19,10833,57,1849,NaN
2,GCF_0232277151_ASM2322771v1_genomic,2359936,33.39,128,0.9771,1.0078,0.9676,1.0127,65,140569,6,43709,15,2348,NaN
3,GCA_0282797051_ASM2827970v1_genomic,1895518,46.75,112,0.8550,1.0268,0.7823,1.0503,88,145438,4,34574,15,2080,NaN
4,GCF_0001452951_ASM14529v1_genomic,1639135,48.64,129,0.9847,1.0078,0.9809,1.0132,2,1634695,1,1634695,1,1750,NaN


In [62]:
mi["genome_size"] = mi["Length"] / mi["Weighted completeness"] 

mi.head()

,Name,Length,GC-content,Present Markers,Completeness,Redundancy,Weighted completeness,Weighted redundancy,Contigs,N50,L50,N90,L90,CDs,Unnamed: 14,genome_size
0,GCF_0005138551_ASM51385v1_genomic,1307099,45.16,126,0.9618,1.0079,0.9651,1.0125,1,1307099,1,1307099,1,1386,NaN,1.354366e+06
1,GCA_0388711751_ASM3887117v1_genomic,1717258,41.66,117,0.8931,1.0256,0.8529,1.0441,80,29394,19,10833,57,1849,NaN,2.013434e+06
2,GCF_0232277151_ASM2322771v1_genomic,2359936,33.39,128,0.9771,1.0078,0.9676,1.0127,65,140569,6,43709,15,2348,NaN,2.438958e+06
3,GCA_0282797051_ASM2827970v1_genomic,1895518,46.75,112,0.8550,1.0268,0.7823,1.0503,88,145438,4,34574,15,2080,NaN,2.423007e+06
4,GCF_0001452951_ASM14529v1_genomic,1639135,48.64,129,0.9847,1.0078,0.9809,1.0132,2,1634695,1,1634695,1,1750,NaN,1.671052e+06


In [63]:
mi["accession"] = mi["Name"].str.extract(r"(GC[AF]_\d{9})")
mi.head()

,Name,Length,GC-content,Present Markers,Completeness,Redundancy,Weighted completeness,Weighted redundancy,Contigs,N50,L50,N90,L90,CDs,Unnamed: 14,genome_size,accession
0,GCF_0005138551_ASM51385v1_genomic,1307099,45.16,126,0.9618,1.0079,0.9651,1.0125,1,1307099,1,1307099,1,1386,NaN,1.354366e+06,GCF_000513855
1,GCA_0388711751_ASM3887117v1_genomic,1717258,41.66,117,0.8931,1.0256,0.8529,1.0441,80,29394,19,10833,57,1849,NaN,2.013434e+06,GCA_038871175
2,GCF_0232277151_ASM2322771v1_genomic,2359936,33.39,128,0.9771,1.0078,0.9676,1.0127,65,140569,6,43709,15,2348,NaN,2.438958e+06,GCF_023227715
3,GCA_0282797051_ASM2827970v1_genomic,1895518,46.75,112,0.8550,1.0268,0.7823,1.0503,88,145438,4,34574,15,2080,NaN,2.423007e+06,GCA_028279705
4,GCF_0001452951_ASM14529v1_genomic,1639135,48.64,129,0.9847,1.0078,0.9809,1.0132,2,1634695,1,1634695,1,1750,NaN,1.671052e+06,GCF_000145295


## gene density

In [64]:
mi["gene_density"] = mi["CDs"] / (mi["Length"] / 1000000) 
mi.head()

,Name,Length,GC-content,Present Markers,Completeness,Redundancy,Weighted completeness,Weighted redundancy,Contigs,N50,L50,N90,L90,CDs,Unnamed: 14,genome_size,accession,gene_density
0,GCF_0005138551_ASM51385v1_genomic,1307099,45.16,126,0.9618,1.0079,0.9651,1.0125,1,1307099,1,1307099,1,1386,NaN,1.354366e+06,GCF_000513855,1060.363446
1,GCA_0388711751_ASM3887117v1_genomic,1717258,41.66,117,0.8931,1.0256,0.8529,1.0441,80,29394,19,10833,57,1849,NaN,2.013434e+06,GCA_038871175,1076.716486
2,GCF_0232277151_ASM2322771v1_genomic,2359936,33.39,128,0.9771,1.0078,0.9676,1.0127,65,140569,6,43709,15,2348,NaN,2.438958e+06,GCF_023227715,994.942236
3,GCA_0282797051_ASM2827970v1_genomic,1895518,46.75,112,0.8550,1.0268,0.7823,1.0503,88,145438,4,34574,15,2080,NaN,2.423007e+06,GCA_028279705,1097.325375
4,GCF_0001452951_ASM14529v1_genomic,1639135,48.64,129,0.9847,1.0078,0.9809,1.0132,2,1634695,1,1634695,1,1750,NaN,1.671052e+06,GCF_000145295,1067.636284


## summary table

In [65]:
#extract relevant columns 

columns = ["accession", "genome_size", "gene_density", "GC-content"]
mi_df = mi[columns]
mi_df.head()

,accession,genome_size,gene_density,GC-content
0,GCF_000513855,1.354366e+06,1060.363446,45.16
1,GCA_038871175,2.013434e+06,1076.716486,41.66
2,GCF_023227715,2.438958e+06,994.942236,33.39
3,GCA_028279705,2.423007e+06,1097.325375,46.75
4,GCF_000145295,1.671052e+06,1067.636284,48.64


In [66]:
#combine by accession

summary = spot_df.merge(mi_df, on = "accession", how="left")
summary.head()


,accession,temperature_optimum,genome_size,gene_density,GC-content
0,GCA_903884505,44.83760079640768,1.759997e+06,1030.709851,56.76
1,GCA_029259185,67.8920121656165,1.350382e+06,1126.294884,51.96
2,GCA_000145985,86.60778627725867,1.942986e+06,1065.591729,35.69
3,GCA_016927425,45.308049681186404,2.358715e+06,979.862129,56.05
4,GCF_029338575,46.352929453610784,4.428875e+06,999.456175,67.92


In [67]:
#save to file
summary.to_csv("summary.tsv", sep="\t", index=False)

## protein features
- protein length
- fraction of charged aa (DEKR)
- fraction of polar amino acids (STNQ)
- fraction of ILVWYGERKP i.e. thermostability group
- hydropathicity (GRAVY)

In [89]:
#simple file parser

def fasta_parser(file):
    with open(file, "r") as f:
        seq = ''
        for line in f:
            if line.startswith(">"):
                if seq:
                    yield seq
                    seq = ''
            else:
                seq += line.strip()
        if seq:
            yield seq

In [90]:
#amino acid feature calculation

def compute_features(faa_file):
    charged = "DEKR"
    polar = "STNQ"
    ilvwyg = "ILVWYGERKP"
    hydrophobic = "AVILMFWY"
    aromatic = "FYW"

    #kyte-doolittle values
    hydropathy = {
        'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
        'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
        'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6,
        'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2} 

    total_aa = 0
    counts = {
        "charged": 0,
        "polar": 0,
        "hydrophobic": 0,
        "aromatic": 0,
        "ilvwyg": 0,
        "gravy_sum": 0.0,
        "protein_count": 0
    }

    for seq in fasta_parser(faa_file):
        seq = seq.upper()
        if not seq:
            continue
        counts["protein_count"] += 1
        total_aa += len(seq)
        for aa in seq:
            if aa in charged: 
                counts["charged"] += 1
            if aa in polar: 
                counts["polar"] += 1
            if aa in hydrophobic: 
                counts["hydrophobic"] += 1
            if aa in aromatic:
                counts["aromatic"] += 1
            if aa in ilvwyg:
                counts["ilvwyg"] += 1
            if aa in hydropathy: 
                counts["gravy_sum"] += hydropathy[aa]

    if total_aa == 0:
        return None

    return {"aa_total": total_aa,
        "proteins": counts["protein_count"],
        "frac_charged": counts["charged"] / total_aa,
        "frac_polar": counts["polar"] / total_aa,
        "frac_hydrophobic": counts["hydrophobic"] / total_aa,
        "frac_aromatic": counts["aromatic"] / total_aa,
        "frac_ilvwyg": counts["ilvwyg"] / total_aa,
        "gravy_mean": counts["gravy_sum"] / total_aa,}


In [91]:
from glob import glob
import pandas as pd
import os

records = []
for faa in glob("prokka_out/prokka_*/*.faa"):
    accession = "_".join(os.path.basename(faa).split(".")[0].split("_")[1:])
    feats = compute_features(faa)
    if feats:
        feats["accession"] = accession
        records.append(feats)

aa_df = pd.DataFrame(records)

In [92]:
aa_df.head()

,aa_total,proteins,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,gravy_mean,accession
0,797456,2863,0.246355,0.190705,0.424899,0.083566,0.575569,-0.137496,GCA_030611265
1,452523,1604,0.247839,0.154291,0.436809,0.073408,0.611953,-0.059902,GCF_000711905
2,678559,2203,0.224437,0.171110,0.436650,0.080867,0.577023,-0.024972,GCF_018687785
3,464455,1720,0.233990,0.187226,0.437080,0.082544,0.591493,-0.071790,GCA_016871995
4,425647,1502,0.246253,0.182870,0.440454,0.089100,0.620850,-0.052428,GCA_001940665


In [110]:
#combine by accession

merged = summary.merge(aa_df, on = "accession", how="left") 
merged.head()

,accession,temperature_optimum,genome_size,gene_density,GC-content,accession_clean_x,aa_total,proteins,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,gravy_mean,accession_clean_y
0,GCA_903884505,44.83760079640768,1.759997e+06,1030.709851,56.76,GCA_903884505,460626.0,1530.0,0.237657,0.175387,0.428979,0.082095,0.579288,-0.131588,GCA_903884505
1,GCA_029259185,67.8920121656165,1.350382e+06,1126.294884,51.96,GCA_029259185,373399.0,1378.0,0.266768,0.148048,0.423065,0.077228,0.641070,-0.140035,GCA_029259185
2,GCA_000145985,86.60778627725867,1.942986e+06,1065.591729,35.69,GCA_000145985,544497.0,1994.0,0.241572,0.168431,0.464671,0.094364,0.635449,0.052572,GCA_000145985
3,GCA_016927425,45.308049681186404,2.358715e+06,979.862129,56.05,GCA_016927425,663102.0,2210.0,0.238681,0.165441,0.432558,0.075074,0.591933,-0.078561,GCA_016927425
4,GCF_029338575,46.352929453610784,4.428875e+06,999.456175,67.92,GCF_029338575,1199013.0,4285.0,0.252032,0.152640,0.428929,0.072433,0.574337,-0.153448,GCF_029338575


It appears as if there is some mismatch between accessions, which needs to be inspected to be fixed

In [113]:
summary["accession"] = summary["accession"].apply(strip_version)
aa_df["accession"] = aa_df["accession"].apply(strip_version)

main_accessions = set(summary["accession"])
aa_accessions = set(aa_df["accession"])

# In main_df but missing from aa_df
missing_in_aa = main_accessions - aa_accessions

# In aa_df but missing from main_df
missing_in_main = aa_accessions - main_accessions

print("Missing in aa_df:")
for acc in sorted(missing_in_aa):
    print(acc)

print("\nMissing in main_df:")
for acc in sorted(missing_in_main):
    print(acc)

Missing in aa_df:
GCA_002009975
GCA_002505765
GCA_003166335
GCA_003584625
GCA_004211975
GCA_013572355
GCA_014859785
GCA_017656505
GCA_019084405
GCA_021159385
GCA_025061095
GCA_027036775
GCA_030055515
GCA_038741495
GCF_000014945
GCF_000063445
GCF_000191585
GCF_000194625
GCF_000404225
GCF_000455345
GCF_000455365
GCF_001484685
GCF_012027325
GCF_017873855
GCF_017874455
GCF_018128925
GCF_023227715
GCF_030464345

Missing in main_df:
A_GCA_003166335
A_GCA_003584625
A_GCA_013572355
A_GCA_014859785
A_GCA_017656505
A_GCA_030055515
A_GCF_000063445
A_GCF_000404225
A_GCF_000455345
A_GCF_001484685
A_GCF_018128925
A_GCF_030464345
B_GCA_025061095
B_GCA_027036775
B_GCF_000014945
B_GCF_000191585
B_GCF_000455365
B_GCF_012027325
B_GCF_017873855
C_GCA_002009975
C_GCA_021159385
C_GCA_038741495
C_GCF_000194625
C_GCF_017874455
D_GCA_002505765
D_GCA_004211975
E_GCF_023227715
F_GCA_019084405


In [130]:
#strip and match the accessions

summary["accession"] = summary["accession"].str.extract(r"(GCA_\d+|GCF_\d+)")
aa_df["accession"] = aa_df["accession"].str.extract(r"(GCA_\d+|GCF_\d+)")
merged = pd.merge(summary, aa_df, on="accession", how="inner")

merged.head()

,accession,temperature_optimum,genome_size,gene_density,GC-content,accession_clean_x,aa_total,proteins,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,gravy_mean,accession_clean_y
0,GCA_903884505,44.83760079640768,1.759997e+06,1030.709851,56.76,GCA_903884505,460626,1530,0.237657,0.175387,0.428979,0.082095,0.579288,-0.131588,GCA_903884505
1,GCA_029259185,67.8920121656165,1.350382e+06,1126.294884,51.96,GCA_029259185,373399,1378,0.266768,0.148048,0.423065,0.077228,0.641070,-0.140035,GCA_029259185
2,GCA_000145985,86.60778627725867,1.942986e+06,1065.591729,35.69,GCA_000145985,544497,1994,0.241572,0.168431,0.464671,0.094364,0.635449,0.052572,GCA_000145985
3,GCA_016927425,45.308049681186404,2.358715e+06,979.862129,56.05,GCA_016927425,663102,2210,0.238681,0.165441,0.432558,0.075074,0.591933,-0.078561,GCA_016927425
4,GCF_029338575,46.352929453610784,4.428875e+06,999.456175,67.92,GCF_029338575,1199013,4285,0.252032,0.152640,0.428929,0.072433,0.574337,-0.153448,GCF_029338575


In [131]:
#rename one to accession and drop
merged["accession"] = merged["accession_clean_x"] 
merged = merged.drop(columns=["accession_clean_x", "accession_clean_y"])

merged.head()

,accession,temperature_optimum,genome_size,gene_density,GC-content,aa_total,proteins,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,gravy_mean
0,GCA_903884505,44.83760079640768,1.759997e+06,1030.709851,56.76,460626,1530,0.237657,0.175387,0.428979,0.082095,0.579288,-0.131588
1,GCA_029259185,67.8920121656165,1.350382e+06,1126.294884,51.96,373399,1378,0.266768,0.148048,0.423065,0.077228,0.641070,-0.140035
2,GCA_000145985,86.60778627725867,1.942986e+06,1065.591729,35.69,544497,1994,0.241572,0.168431,0.464671,0.094364,0.635449,0.052572
3,GCA_016927425,45.308049681186404,2.358715e+06,979.862129,56.05,663102,2210,0.238681,0.165441,0.432558,0.075074,0.591933,-0.078561
4,GCF_029338575,46.352929453610784,4.428875e+06,999.456175,67.92,1199013,4285,0.252032,0.152640,0.428929,0.072433,0.574337,-0.153448


In [132]:
#save as file
merged.to_csv("summary.tsv", sep="\t", index=False)

## **tRNA features**
- GC%
- average tRNA length

In [26]:
import glob
import pandas as pd

#find all files
fna_files = glob.glob("trna_out/*_trna.fna")

rows = []
lengths = []

for fna_file in fna_files:
    accession = fna_file.split("/")[-1].replace("_trna.fna", "")

    gc_values = []
    with open(fna_file) as f:
        seq = ""
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if seq:
                    gc = (seq.count("G") + seq.count("C")) / len(seq) * 100
                    gc_values.append(gc)
                    lengths.append(len(seq))
                    seq = ""
            else:
                seq += line.strip()
        if seq != "":
            lengths.append(len(seq))
                
        #processing last sequence
        if seq:
            gc = (seq.count("G") + seq.count("C")) / len(seq) * 100
            gc_values.append(gc)

    avg_length = sum(lengths) / len(lengths) if lengths else 0
    #calculating mean GC%
    if gc_values:
        mean_gc = sum(gc_values) / len(gc_values)
    else:
         mean_gc = None 
    
    rows.append({"accession": accession, "tRNA_GC_mean": mean_gc, "avg_tRNA_length": avg_length})

#save results as tsv
df = pd.DataFrame(rows)
df.head()

,accession,tRNA_GC_mean,avg_tRNA_length
0,GCA_020854815,61.549369,78.978723
1,GCA_029982165,61.483544,81.010753
2,GCA_029887765,63.403735,82.769784
3,GCA_018814355,60.239999,84.005464
4,GCA_015520265,71.878877,86.924107


In [30]:
summary = pd.read_csv("summary.tsv", sep="\t")
summary.head()

,accession,temperature_optimum,genome_size,gene_density,GC-content,aa_total,proteins,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,gravy_mean
0,GCA_903884505,44.837601,1.759997e+06,1030.709851,56.76,460626,1530,0.237657,0.175387,0.428979,0.082095,0.579288,-0.131588
1,GCA_029259185,67.892012,1.350382e+06,1126.294884,51.96,373399,1378,0.266768,0.148048,0.423065,0.077228,0.641070,-0.140035
2,GCA_000145985,86.607786,1.942986e+06,1065.591729,35.69,544497,1994,0.241572,0.168431,0.464671,0.094364,0.635449,0.052572
3,GCA_016927425,45.308050,2.358715e+06,979.862129,56.05,663102,2210,0.238681,0.165441,0.432558,0.075074,0.591933,-0.078561
4,GCF_029338575,46.352929,4.428875e+06,999.456175,67.92,1199013,4285,0.252032,0.152640,0.428929,0.072433,0.574337,-0.153448


In [31]:
merged = pd.merge(summary, df, on="accession", how="inner")

merged.head()

,accession,temperature_optimum,genome_size,gene_density,GC-content,aa_total,proteins,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,gravy_mean,tRNA_GC_mean,avg_tRNA_length
0,GCA_903884505,44.837601,1.759997e+06,1030.709851,56.76,460626,1530,0.237657,0.175387,0.428979,0.082095,0.579288,-0.131588,59.950459,90.292916
1,GCA_029259185,67.892012,1.350382e+06,1126.294884,51.96,373399,1378,0.266768,0.148048,0.423065,0.077228,0.641070,-0.140035,64.962643,89.959412
2,GCA_000145985,86.607786,1.942986e+06,1065.591729,35.69,544497,1994,0.241572,0.168431,0.464671,0.094364,0.635449,0.052572,69.203992,88.351724
3,GCA_016927425,45.308050,2.358715e+06,979.862129,56.05,663102,2210,0.238681,0.165441,0.432558,0.075074,0.591933,-0.078561,61.710844,90.439763
4,GCF_029338575,46.352929,4.428875e+06,999.456175,67.92,1199013,4285,0.252032,0.152640,0.428929,0.072433,0.574337,-0.153448,61.729627,88.052546


In [32]:
#save as file
merged.to_csv("summary.tsv", sep="\t", index=False)

## **codonw features**
- GC3s
- Gravy
- aromo
- Nc 

In [8]:
import pandas as pd
import glob

#collect .out files
out_files = glob.glob("gc3_results/*_cds.out")

all_features = []

for out_file in out_files: 
    accession = out_file.split("/")[-1].replace("_cds.out", "")

    with open(out_file) as file:
        lines = []
        for line in file: 
            line = line.strip() 
            if line != "":
                lines.append(line) 
    #column names
    header_line = lines[0]
    header = header_line.split()

    #data values
    value_line = lines[1]
    values = value_line.split()

    #make dictionary (column:value) 
    feature={}
    for i in range(len(header)): 
        feature[header[i]]=values[i]
    
    feature['accession']=accession

    #save genome's features
    all_features.append(feature) 


#convert to dataframe
codonw_df = pd.DataFrame(all_features) 
codonw_df.head()

,title,T3s,C3s,A3s,G3s,CAI,CBI,Fop,Nc,GC3s,GC,L_sym,L_aa,Gravy,Aromo,accession
0,Average_of_genes,0.0764,0.5981,0.1059,0.4366,0.244,0.195,0.531,36.97,0.849,0.664,1275432,1313095,-0.244439,0.073193,GCF_023238205
1,Average_of_genes,0.2393,0.3610,0.2916,0.3512,0.180,0.031,0.426,54.42,0.563,0.501,460714,477945,-0.072605,0.082934,GCA_038852285
2,Average_of_genes,0.3606,0.2477,0.4140,0.2868,0.193,-0.024,0.407,47.92,0.393,0.395,374724,386831,-0.234110,0.093312,GCA_018303625
3,Average_of_genes,0.3754,0.2306,0.3994,0.2419,0.170,-0.027,0.391,52.45,0.367,0.421,543397,564529,-0.031203,0.094495,GCA_018396755
4,Average_of_genes,0.1751,0.4052,0.2041,0.4327,0.161,0.002,0.405,45.86,0.679,0.563,496116,513689,0.065056,0.080928,GCA_015521325


In [9]:
#extract relevant features

columns = ["accession", "GC3s", "Gravy", "Aromo", "Nc"]

feature_df = codonw_df[columns]
feature_df.head()

,accession,GC3s,Gravy,Aromo,Nc
0,GCF_023238205,0.849,-0.244439,0.073193,36.97
1,GCA_038852285,0.563,-0.072605,0.082934,54.42
2,GCA_018303625,0.393,-0.234110,0.093312,47.92
3,GCA_018396755,0.367,-0.031203,0.094495,52.45
4,GCA_015521325,0.679,0.065056,0.080928,45.86


In [11]:
#load summary.tsv

summary = pd.read_csv("summary.tsv", sep="\t")

summary.head()

,accession,temperature_optimum,genome_size,gene_density,GC-content,aa_total,proteins,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,gravy_mean,tRNA_GC_mean,avg_tRNA_length
0,GCA_903884505,44.837601,1.759997e+06,1030.709851,56.76,460626,1530,0.237657,0.175387,0.428979,0.082095,0.579288,-0.131588,59.950459,90.292916
1,GCA_029259185,67.892012,1.350382e+06,1126.294884,51.96,373399,1378,0.266768,0.148048,0.423065,0.077228,0.641070,-0.140035,64.962643,89.959412
2,GCA_000145985,86.607786,1.942986e+06,1065.591729,35.69,544497,1994,0.241572,0.168431,0.464671,0.094364,0.635449,0.052572,69.203992,88.351724
3,GCA_016927425,45.308050,2.358715e+06,979.862129,56.05,663102,2210,0.238681,0.165441,0.432558,0.075074,0.591933,-0.078561,61.710844,90.439763
4,GCF_029338575,46.352929,4.428875e+06,999.456175,67.92,1199013,4285,0.252032,0.152640,0.428929,0.072433,0.574337,-0.153448,61.729627,88.052546


In [12]:
#merge dfs

merged = pd.merge(summary, feature_df, on="accession", how="inner")

merged.head()

,accession,temperature_optimum,genome_size,gene_density,GC-content,aa_total,proteins,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,gravy_mean,tRNA_GC_mean,avg_tRNA_length,GC3s,Gravy,Aromo,Nc
0,GCA_903884505,44.837601,1.759997e+06,1030.709851,56.76,460626,1530,0.237657,0.175387,0.428979,0.082095,0.579288,-0.131588,59.950459,90.292916,0.726,-0.130644,0.082074,45.67
1,GCA_029259185,67.892012,1.350382e+06,1126.294884,51.96,373399,1378,0.266768,0.148048,0.423065,0.077228,0.641070,-0.140035,64.962643,89.959412,0.594,-0.138000,0.077224,53.88
2,GCA_000145985,86.607786,1.942986e+06,1065.591729,35.69,544497,1994,0.241572,0.168431,0.464671,0.094364,0.635449,0.052572,69.203992,88.351724,0.241,0.054511,0.094426,41.70
3,GCA_016927425,45.308050,2.358715e+06,979.862129,56.05,663102,2210,0.238681,0.165441,0.432558,0.075074,0.591933,-0.078561,61.710844,90.439763,0.686,-0.077339,0.075046,49.23
4,GCF_029338575,46.352929,4.428875e+06,999.456175,67.92,1199013,4285,0.252032,0.152640,0.428929,0.072433,0.574337,-0.153448,61.729627,88.052546,0.898,-0.152036,0.072411,33.59


In [13]:
#save as file
merged.to_csv("summary.tsv", sep="\t", index=False)

## additional features

In [5]:
#load file

import pandas as pd 

clean_file = pd.read_csv("summary_clean.tsv", sep="\t")
clean_file.head()

,accession,ogt,genome_size,gene_density,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,trna_gc,trna_length,gc3s,gravy,aromo,nc,ratio_charged_polar
0,GCA_903884505,44.84,1.76,1030.71,0.238,0.175,0.429,0.082,0.579,59.95,90.29,72.6,-0.131,0.082,45.67,1.360000
1,GCA_029259185,67.89,1.35,1126.29,0.267,0.148,0.423,0.077,0.641,64.96,89.96,59.4,-0.138,0.077,53.88,1.804054
2,GCA_000145985,86.61,1.94,1065.59,0.242,0.168,0.465,0.094,0.635,69.20,88.35,24.1,0.055,0.094,41.70,1.440476
3,GCA_016927425,45.31,2.36,979.86,0.239,0.165,0.433,0.075,0.592,61.71,90.44,68.6,-0.077,0.075,49.23,1.448485
4,GCF_029338575,46.35,4.43,999.46,0.252,0.153,0.429,0.072,0.574,61.73,88.05,89.8,-0.152,0.072,33.59,1.647059


In [11]:
#add ratio of frac_charged / frac_polar

clean_file["ratio_charged_polar"] = clean_file["frac_charged"] / clean_file["frac_polar"]

#round number 
clean_file = clean_file.round({"ratio_charged_polar" : 3})

clean_file.head() 

,accession,ogt,genome_size,gene_density,frac_charged,frac_polar,frac_hydrophobic,frac_aromatic,frac_ilvwyg,trna_gc,trna_length,gc3s,gravy,aromo,nc,ratio_charged_polar
0,GCA_903884505,44.84,1.76,1030.71,0.238,0.175,0.429,0.082,0.579,59.95,90.29,72.6,-0.131,0.082,45.67,1.360
1,GCA_029259185,67.89,1.35,1126.29,0.267,0.148,0.423,0.077,0.641,64.96,89.96,59.4,-0.138,0.077,53.88,1.804
2,GCA_000145985,86.61,1.94,1065.59,0.242,0.168,0.465,0.094,0.635,69.20,88.35,24.1,0.055,0.094,41.70,1.440
3,GCA_016927425,45.31,2.36,979.86,0.239,0.165,0.433,0.075,0.592,61.71,90.44,68.6,-0.077,0.075,49.23,1.448
4,GCF_029338575,46.35,4.43,999.46,0.252,0.153,0.429,0.072,0.574,61.73,88.05,89.8,-0.152,0.072,33.59,1.647


In [10]:
#save as file
clean_file.to_csv("summary_clean.tsv", sep="\t", index=False)